# Extract Features using Transfer Learning Densenet

This notebook contains the code of the feature extraction using DenseNet121 from Tensorflow Keras API. As an output, we obtained feature vector with the dimesnions of N x 1024 where N is the number of samples.

## Libraries

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from keras.preprocessing.image import ImageDataGenerator
import pandas as pd
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from keras.layers import Activation, Dropout, Flatten, Dense, GlobalMaxPooling2D, GlobalAveragePooling2D
import os
import cv2 as cv
from PIL import Image
import os
import pickle
import cv2
from tensorflow.keras.utils import normalize
from PIL import Image
import glob
import numpy as np
plt.style.use('classic')
np.warnings.filterwarnings('ignore', category=np.VisibleDeprecationWarning) 

from keras.models import Model, Sequential
from keras.layers import Dense, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.layers import BatchNormalization
import os
from sklearn.preprocessing import StandardScaler
import seaborn as sns
from keras.applications.vgg16 import VGG16
from sklearn.pipeline import make_pipeline
from sklearn.svm import SVC
import tensorflow as tf
from tqdm import tqdm
import pickle

## Function to Obtain Feature vector of a given path of images




In [ ]:
# Image paths
paths_ml = pickle.load(open('/content/drive/MyDrive/Ars_machinae_autodiscentis/Inceptum/paths.p','rb'))

In [ ]:
def dense_model(image):

  image = tf.keras.applications.densenet.preprocess_input(image)
  SIZE=460
  densenet = tf.keras.applications.DenseNet121(weights='imagenet', include_top=False, input_shape=(SIZE, SIZE, 3))

  for layer in densenet.layers:
    layer.trainable = False
  
  densenet.summary()
  model = Sequential()
  model.add(densenet)
  model.add(GlobalAveragePooling2D())

  feature_extractor = model.predict(image)
  features = feature_extractor.reshape(feature_extractor.shape[0], -1)
  X_for_RF_d = features 
  return X_for_RF_d

In [ ]:
def read_images(paths_ml):
  images = []
  for path in tqdm(paths_ml):
    im = cv.imread(path)
    im = cv.resize(im, (460,460))
    images.append(im)
  return images

In [ ]:
def get_file_fv(paths,feature_name,fold, train_test, magnification):
  mag_dictionary = {0:'40',1:'100',2:'200',3:'400'}
  train_dic = {0:'train',1:'test'}
  images = np.array(read_images(paths[fold,train_test,magnification]))
  fv = dense_model(images)
  os.chdir('/content/drive/MyDrive/Ars_machinae_autodiscentis/Inceptum/Machine_Learning/pickle_files')
  pickle.dump(fv ,open(f"{feature_name}_f{fold+1}_{train_dic[train_test]}_{mag_dictionary[magnification]}x_fv.p","wb"))

In [ ]:
for fold in range(0,5):
   for tt in range(2):
     for mag in range(4):
       get_file_fv(paths=paths_ml,feature_name = 'dense',fold=fold,train_test=tt,magnification=mag)

100%|██████████| 717/717 [02:29<00:00,  4.79it/s]


In [ ]:
os.chdir('/content/drive/MyDrive/Ars_machinae_autodiscentis/Inceptum/Machine_Learning/pickle_files')
pickle.dump(pd.DataFrame(vector_f1_40_train),open(f"dense_f1_train_40x_fv.p","wb"))

In [ ]:
os.chdir('/content/drive/MyDrive/Ars_machinae_autodiscentis/Inceptum/Machine_Learning/pickle_files')
pickle.dump(pd.DataFrame(vector_f1_40_test),open(f"dense_f1_test_40x_fv.p","wb"))